In [31]:
from bs4 import BeautifulSoup as bs

import requests
import re

import pyodbc

import sqlalchemy as sal
from sqlalchemy import create_engine
from sqlalchemy.sql import text

import pandas as pd
import numpy as np
import pyodbc

from datetime import datetime
import time
import random

import os

In [1]:
schedule_url = 'https://www.cbssports.com/nfl/news/2024-nfl-team-by-team-schedules-times-dates-tv-for-the-entire-season-for-all-32-franchises/'
schedule_url

'https://www.cbssports.com/nfl/news/2024-nfl-team-by-team-schedules-times-dates-tv-for-the-entire-season-for-all-32-franchises/'

In [3]:
#get team table
teams_schedule_2024_li = pd.read_html(schedule_url, attrs={'class':'TableBuilder'})

In [4]:
# delay execute so not to trigger ip address block
time.sleep(random.randrange(7,9))

In [5]:
#get team name
def webscrape_team_name (url:str):

    html = requests.get(url).text

    soup = bs(html, 'html.parser')

    h2_team_names_li = soup.find_all('h2')[1:]

    team_li = []
    
    for h2 in h2_team_names_li:
        try:
            team = h2.next_element.contents[0].split()[-1]

        except:
            team = h2.contents[0].split()[-1]

        team_li.append(team)

    return team_li

In [6]:
team_li = webscrape_team_name(schedule_url)
team_li

['Cardinals',
 'Falcons',
 'Ravens',
 'Bills',
 'Panthers',
 'Bears',
 'Bengals',
 'Browns',
 'Cowboys',
 'Broncos',
 'Lions',
 'Packers',
 'Texans',
 'Colts',
 'Jaguars',
 'Chiefs',
 'Raiders',
 'Chargers',
 'Angeles',
 'Dolphins',
 'Vikings',
 'Patriots',
 'Saints',
 'Giants',
 'Jets',
 'Eagles',
 'Steelers',
 '49ers',
 'Seahawks',
 'Buccaneers',
 'Titans',
 'Commanders']

In [24]:
year = 2024

for df, team in zip(teams_schedule_2024_li, team_li):
    #add year column
    df['Year'] = [year] * len(df)

   #add team column
    df['Team'] = [team] * len(df)

    #opponent name
    opp_dict = {'Opponent': []}
    for i in range(0, len(df)):
        opp = df['Matchup'].iloc[i].split()[-1]

        opp_dict['Opponent'].append(opp)
        
    df['Opponent'] = opp_dict['Opponent']

    df.to_csv(f'NFL_Data_2024/{team}/Schedule')

In [26]:
pd.read_csv('NFL_Data_2024/49ers/Schedule', index_col=0)

,Date,Matchup,Time (ET),TV,Year,Team,Opponent
0,9/9/2024,San Francisco 49ers vs. New York Jets,8:15p,ESPN/ABC,2024,49ers,Jets
1,9/15/2024,San Francisco 49ers at Minnesota Vikings,1:00p,CBS,2024,49ers,Vikings
2,9/22/2024,San Francisco 49ers at Los Angeles Rams,4:25p,FOX,2024,49ers,Rams
3,9/29/2024,San Francisco 49ers vs. New England Patriots,4:05p,FOX,2024,49ers,Patriots
4,10/6/2024,San Francisco 49ers vs. Arizona Cardinals,4:05p,FOX,2024,49ers,Cardinals
5,10/10/2024,San Francisco 49ers at Seattle Seahawks,8:15p,Prime Video,2024,49ers,Seahawks
6,10/20/2024,San Francisco 49ers vs. Kansas City Chiefs,4:25p,FOX,2024,49ers,Chiefs
7,10/27/2024,San Francisco 49ers vs. Dallas Cowboys,8:20p,NBC*,2024,49ers,Cowboys
8,NaN,Week 9 bye,NaN,NaN,2024,49ers,bye
9,11/10/2024,San Francisco 49ers at Tampa Bay Buccaneers,1:00p,FOX,2024,49ers,Buccaneers


In [27]:
#connect schedule to sql
# union of all season and team to one table for each table
servername = '****'
football_data = 'Football_Data'
trusted_conneciton = '?trusted_conneciton=yes'
driver = '&driver=ODBC+Driver+17+for+SQL+Server'
username = 'MAKeith92'
password = 'Lopez!123'

In [32]:
#create engine url to connect to sql server
engine_football_data = create_engine(f'mssql+pyodbc://@{servername}/{football_data}{trusted_conneciton}{driver}')
engine_football_data

Engine(mssql+pyodbc://@DESKTOP-5IAPFQC/Football_Data?driver=ODBC+Driver+17+for+SQL+Server&trusted_conneciton=yes)

In [33]:
conn_engine_football_data = engine_football_data.connect()

In [36]:
teams_schedule_df = pd.DataFrame()

for team in team_li:

    df = pd.read_csv(f'NFL_Data_2024/{team}/Schedule', index_col=0)

    teams_schedule_df = pd.concat([teams_schedule_df, df], axis=0)

In [42]:
teams_schedule_df.reset_index(inplace=True, drop=True)
teams_schedule_df

,Date,Matchup,Time (ET),TV,Year,Team,Opponent
0,9/8/2024,Arizona Cardinals at Buffalo Bills,1:00p,CBS,2024,Cardinals,Bills
1,9/15/2024,Arizona Cardinals vs. Los Angeles Rams,4:05p,FOX,2024,Cardinals,Rams
2,9/22/2024,Arizona Cardinals vs. Detroit Lions,4:25p,FOX,2024,Cardinals,Lions
3,9/29/2024,Arizona Cardinals vs. Washington Commanders,4:05p,FOX,2024,Cardinals,Commanders
4,10/6/2024,Arizona Cardinals at San Francisco 49ers,4:05p,FOX,2024,Cardinals,49ers
...,...,...,...,...,...,...,...
571,NaN,Week 14 bye,NaN,NaN,2024,Commanders,bye
572,12/15/2024,Washington Commanders at New Orleans Saints,1:00p,FOX,2024,Commanders,Saints
573,12/22/2024,Washington Commanders vs. Philadelphia Eagles,1:00p,FOX,2024,Commanders,Eagles
574,12/28/2024,Washington Commanders vs. Atlanta Falcons,TBD,TBD,2024,Commanders,Falcons


In [43]:
teams_schedule_df.to_sql('2024_Schedule', con=conn_engine_football_data, if_exists='replace')

52